# Train and Deploy an Income Predictor Model Using Flask

<b> Download and load the dataset </b>

In [1]:
import pandas as pd
import pickle
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
url = 'https://raw.githubusercontent.com/PacktWorkshops/The-Data-Science-Workshop/master/Chapter18/Dataset/phpMawTba.csv'

In [3]:
df = pd.read_csv(url)
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,Female,0,0,30,United-States,<=50K


<b> Extract the response variable </b>

In [4]:
y = df.pop('class')

<b> Split the dataset into training and test sets </b>

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.33, random_state=8)

<b> Extract the list of categories for each categorical column </b>

In [6]:
cat_columns = [col for col in df.columns if df[col].dtype == 'object']
cat_columns

['workclass',
 'education',
 'marital-status',
 'occupation',
 'relationship',
 'sex',
 'native-country']

<b> Save the list of categories and categorical column names into files </b>

In [7]:
column_cat = {}

In [8]:
for col in cat_columns:
    column_cat[col] = X_train[col].astype('category').cat.categories

In [9]:
pickle.dump(column_cat, open("categories_data.pkl", "wb"))
pickle.dump(cat_columns, open("categorical_columns.pkl", "wb"))

In [10]:
def apply_categories(input_df, cat_dict):
    from pandas.api.types import CategoricalDtype
    for col, cat in cat_dict.items():
        input_df[col] = input_df[col].astype(CategoricalDtype(categories=cat))
    return input_df

In [11]:
X_train_cat = apply_categories(X_train, column_cat)
X_train_cat.dtypes

C:\Users\Hevans\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


age                  int64
workclass         category
fnlwgt               int64
education         category
education-num        int64
marital-status    category
occupation        category
relationship      category
sex               category
capital-gain         int64
capital-loss         int64
hours-per-week       int64
native-country    category
dtype: object

<b> Perform one-hot encoding on categorical variables </b>

In [12]:
X_train_final = pd.get_dummies(X_train_cat, columns=cat_columns)

<b> Train a RandomForest to predict the binary outcome </b>

In [13]:
rf_model = RandomForestClassifier(random_state=8)
rf_model.fit(X_train_final, y_train)

RandomForestClassifier(random_state=8)

<b> Save the trained model into a file </b>

In [14]:
joblib.dump(rf_model, "model.pkl")

['model.pkl']

<b> Create a Flask app </b>

In [15]:
import socket
import threading
import requests
import json
from flask import Flask, jsonify, request
import numpy as np

In [16]:
app = Flask(__name__)

In [17]:
trained_model = joblib.load("model.pkl")
var_means = pickle.load(open("categories_data.pkl", "rb"))
cat_cols = pickle.load(open("categorical_columns.pkl", "rb"))

<b> Create an API endpoint that will perform the same data transformation as for the training set and predict the outcome for a single record </b>

In [18]:
@app.route('/api', methods=['POST'])
def predict():
    data = request.get_json()
    df_test = pd.DataFrame(data, index=[0])
    df_test_clean = apply_categories(df_test, \
                                     var_means)
    df_test_final = pd.get_dummies(df_test_clean, \
                                   columns=cat_cols)
    prediction = trained_model.predict(df_test_final)
    str_pred = np.array2string(prediction)
    return jsonify(str_pred)

In [19]:
flask_thread = threading.Thread(target=app.run, kwargs={'host':'0.0.0.0', 'port':80})
flask_thread.start()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:80/ (Press CTRL+C to quit)


In [20]:
record = X_test.iloc[0,].to_json()
record

'{"age":51,"workclass":" Private","fnlwgt":106151,"education":" 11th","education-num":7,"marital-status":" Divorced","occupation":" Transport-moving","relationship":" Own-child","sex":" Male","capital-gain":0,"capital-loss":0,"hours-per-week":40,"native-country":" United-States"}'

In [21]:
headers = {'content-type': 'application/json', \
           'Accept-Charset': 'UTF-8'}
ip_address = socket.gethostbyname(socket.gethostname())

<b> Send a request to this endpoint </b>

In [22]:
r = requests.post(f"http://{ip_address}/api", data=record, headers=headers)
r.text

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>500 Internal Server Error</title>\n<h1>Internal Server Error</h1>\n<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>\n'